In [8]:
import os
import shutil
import subprocess
import logging
from time import time
import multiprocessing
from pprint import pprint
import collections

import json
import pydicom
import numpy as np
from scipy.sparse import csc_matrix
import math
import matplotlib.pyplot as plt
import numpy.matlib
from scipy import interpolate
from datetime import datetime

from pylinac.core.mtf import MTF
from pylinac.core.utilities import open_path
from pylinac.core import image
from pylinac.core.geometry import Point, Rectangle, Circle
from pylinac.core.io import get_url, retrieve_demo_file
from pylinac.core.profile import CollapsedCircleProfile
from pylinac.core.roi import LowContrastDiskROI, HighContrastDiskROI, bbox_center
from pylinac.core import pdf
from pylinac.core import geometry


import os.path as osp
import tempfile
from unittest import TestCase

import matplotlib.pyplot as plt
import numpy as np

 


from pylinac.planar_imaging import ImagePhantomBase


class CustomPhantom(ImagePhantomBase):
    _demo_filename = 'lasvegas.dcm'
    common_name = 'Las Vegas'
    _phantom_ski_region = None
    phantom_outline_object = {'Rectangle': {'width ratio': 0.62, 'height ratio': 0.62}}
    low_contrast_background_roi_settings = {
        'roi 1': {'distance from center': 0.24, 'angle': 0, 'roi radius': 0.03},
        'roi 2': {'distance from center': 0.24, 'angle': 90, 'roi radius': 0.03},
        'roi 3': {'distance from center': 0.24, 'angle': 180, 'roi radius': 0.03},
        'roi 4': {'distance from center': 0.24, 'angle': 270, 'roi radius': 0.03},
    }
    low_contrast_roi_settings = {
        'roi 1': {'distance from center': 0.107, 'angle': 0.5, 'roi radius': 0.028},
        'roi 2': {'distance from center': 0.141, 'angle': 39.5, 'roi radius': 0.028},
        'roi 3': {'distance from center': 0.205, 'angle': 58, 'roi radius': 0.028},
        'roi 4': {'distance from center': 0.179, 'angle': -76.5, 'roi radius': 0.016},
        'roi 5': {'distance from center': 0.095, 'angle': -63.5, 'roi radius': 0.016},
        'roi 6': {'distance from center': 0.042, 'angle': 0.5, 'roi radius': 0.016},
        'roi 7': {'distance from center': 0.097, 'angle': 65.5, 'roi radius': 0.016},
        'roi 8': {'distance from center': 0.178, 'angle': 76.5, 'roi radius': 0.016},
        'roi 9': {'distance from center': 0.174, 'angle': -97.5, 'roi radius': 0.012},
        'roi 10': {'distance from center': 0.088, 'angle': -105.5, 'roi radius': 0.012},
        'roi 11': {'distance from center': 0.024, 'angle': -183.5, 'roi radius': 0.012},
        'roi 12': {'distance from center': 0.091, 'angle': 105.5, 'roi radius': 0.012},
        'roi 13': {'distance from center': 0.179, 'angle': 97.5, 'roi radius': 0.012},
        'roi 14': {'distance from center': 0.189, 'angle': -113.5, 'roi radius': 0.007},
        'roi 15': {'distance from center': 0.113, 'angle': -131.5, 'roi radius': 0.007},
        'roi 16': {'distance from center': 0.0745, 'angle': -181.5, 'roi radius': 0.007},
        'roi 17': {'distance from center': 0.115, 'angle': 130, 'roi radius': 0.007},
        'roi 18': {'distance from center': 0.191, 'angle': 113, 'roi radius': 0.007},
        'roi 19': {'distance from center': 0.2085, 'angle': -124.6, 'roi radius': 0.003},
        'roi 20': {'distance from center': 0.146, 'angle': -144.3, 'roi radius': 0.003},
    }

    @staticmethod
    def run_demo():
        """Run the Las Vegas phantom analysis demonstration."""
        lv = LasVegas.from_demo_image()
        lv.analyze()
        lv.plot_analyzed_image()

    def _preprocess(self):
        self._check_direction()

    def _check_inversion(self):
        """Check the inversion by using the histogram of the phantom region"""
        roi = self._phantom_ski_region_calc()
        phantom_array = self.image.array[roi.bbox[0]:roi.bbox[2], roi.bbox[1]:roi.bbox[3]]
        phantom_sub_image = image.load(phantom_array)
        phantom_sub_image.crop(int(phantom_sub_image.shape[0]*0.1))
        p5 = np.percentile(phantom_sub_image, 0.5)
        p50 = np.percentile(phantom_sub_image, 50)
        p95 = np.percentile(phantom_sub_image, 99.5)
        dist_to_5 = abs(p50 - p5)
        dist_to_95 = abs(p50 - p95)
        if dist_to_5 > dist_to_95:
            self.image.invert()

    def _check_direction(self) -> None:
        """Check that the phantom is facing the right direction and if not perform a left-right flip of the array."""
        circle = CollapsedCircleProfile(self.phantom_center, self.phantom_radius * 0.175, self.image, ccw=False,
                                        width_ratio=0.16, num_profiles=5)
        roll_amount = np.where(circle.values == circle.values.min())[0][0]
        circle.roll(roll_amount)
        circle.filter(size=0.015, kind='median')
        valleys = circle.find_peaks(max_number=2, kind='value')
        if valleys[0] > valleys[1]:
            self.image.array = np.fliplr(self.image.array)
            self._phantom_ski_region = None

    def _phantom_center_calc(self) -> Point:
        return bbox_center(self._phantom_ski_region_calc())

    def _phantom_radius_calc(self) -> float:
        return self._phantom_ski_region_calc().major_axis_length

    def _phantom_angle_calc(self) -> float:
        return 0.0

    def _phantom_ski_region_calc(self):
        """The skimage region of the phantom outline."""
        if self._phantom_ski_region is not None:
            return self._phantom_ski_region
        else:
            regions = self._get_canny_regions()
            blobs = []
            phantom_bbox_size_mm2 = 20260
            phantom_size_pix = phantom_bbox_size_mm2 * (self.image.dpmm ** 2)
            for phantom_idx, region in enumerate(regions):
                if region.bbox_area < 50:
                    continue
                is_near_iso = np.isclose(region.bbox_area, phantom_size_pix, rtol=0.05)
                is_on_panel = np.isclose(region.bbox_area, phantom_size_pix/2, rtol=0.05)
                if (not is_near_iso) and (not is_on_panel):
                    continue
                hollow = region.extent < 0.02
                near_center_y = np.isclose(region.centroid[0], self.image.center.y, rtol=0.1)
                near_center_x = np.isclose(region.centroid[1], self.image.center.x, rtol=0.1)
                if (is_near_iso or is_on_panel) and near_center_x and near_center_y and hollow:
                    blobs.append(phantom_idx)

            if not blobs or (len(blobs) != 1):
                raise ValueError("Unable to find the Las Vegas phantom in the image. Either the phantom 1) was not centered, 2) was not set at iso (e.g. it's directly on the panel), 3) has an artifact (e.g. couch, rails), or 4) is rotated (not at 0, 90, 180, 270).")

            self._phantom_ski_region = regions[blobs[0]]
            return regions[blobs[0]]



leeds = CustomPhantom('dcm/ACR.dcm')
leeds.analyze(0.01,0.5)
leeds.plot_analyzed_image()
file_out = 'dcm/col.png'
leeds.save_analyzed_image(file_out)

print('http://192.168.10.195:1515/view/work/pylinac/kv_phantom/'+file_out)
pprint(leeds.low_contrast_rois[1].contrast)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(leeds.low_contrast_rois)

NotImplementedError: this transfer syntax JPEG 2000 Image Compression (Lossless Only), can not be read because Pillow lacks the jpeg 2000 decoder plugin

Tutorial

https://github.com/jrkerns/pylinac/blob/a9fd2a24eab617af4cf134fe2c5c09e45fad8119/pylinac/planar_imaging.py

https://github.com/jrkerns/pylinac/blob/a9fd2a24eab617af4cf134fe2c5c09e45fad8119/docs/source/planar_imaging.rst

https://cyberqual.it/autopia-download_en.html